![Darik's School to Code Good](https://raw.githubusercontent.com/darikoneil/dariks_school_to_code_good/dev/logo.png)

# Darik's School to Code Good
### Writing clean, readable scientific Python


This notebook is a practical tour of **readability-first** scientific coding.

## Learning goals
By the end, you should be able to:
- Choose names that communicate intent (and reduce bugs).
- Refactor "one big function" into **small, testable pieces**.
- Recognize and reduce **cyclomatic complexity**.
- Write docstrings that make your future self (and labmates) happy.
- Add lightweight input validation and helpful errors.
- Practice the **test-driven development** loop on small scientific functions.

> Teaching stance: prioritize code that is easy to read and hard to misuse.


## Imports and a tiny simulated dataset
We'll use a synthetic calcium-imaging-like dataset so the notebook runs anywhere.


In [ ]:
%matplotlib inline
from __future__ import annotations

from dataclasses import dataclass
from typing import Iterable, Tuple

import numpy as np
import matplotlib.pyplot as plt

# Reproducibility for demos
rng = np.random.default_rng(7)


In [ ]:
def simulate_calcium_data(
    *,
    num_neurons: int = 40,
    duration_s: float = 60.0,
    frame_rate_hz: float = 20.0,
    baseline: float = 0.0,
    noise_sd: float = 0.15,
    event_rate_hz: float = 0.12,
    tau_decay_s: float = 1.2,
    rng: np.random.Generator | None = None,
) -> tuple[np.ndarray, np.ndarray]:
    """Simulate calcium traces with sparse events convolved by an exponential kernel.

    Parameters
    ----------
    num_neurons:
        Number of simulated neurons (columns in the returned fluorescence array).
    duration_s:
        Recording duration in seconds.
    frame_rate_hz:
        Sampling rate (frames per second).
    baseline:
        Additive baseline offset.
    noise_sd:
        Standard deviation of additive Gaussian noise.
    event_rate_hz:
        Poisson event rate per neuron (events / second).
    tau_decay_s:
        Exponential decay time constant.
    rng:
        Optional NumPy RNG for reproducibility.

    Returns
    -------
    timestamps_s:
        Array of shape (num_samples,) containing timestamps in seconds.
    fluorescence:
        Array of shape (num_samples, num_neurons).
    """
    if rng is None:
        rng = np.random.default_rng()

    if num_neurons <= 0:
        raise ValueError("num_neurons must be positive")
    if duration_s <= 0 or frame_rate_hz <= 0:
        raise ValueError("duration_s and frame_rate_hz must be positive")
    if tau_decay_s <= 0:
        raise ValueError("tau_decay_s must be positive")
    if noise_sd < 0 or event_rate_hz < 0:
        raise ValueError("noise_sd and event_rate_hz must be non-negative")

    num_samples = int(np.round(duration_s * frame_rate_hz))
    timestamps_s = np.arange(num_samples) / frame_rate_hz

    # Event train: Bernoulli approximation to Poisson per frame.
    p_event = event_rate_hz / frame_rate_hz
    events = rng.random((num_samples, num_neurons)) < p_event
    events = events.astype(float)

    # Exponential decay kernel (~10 taus long)
    kernel_len = int(np.ceil(10 * tau_decay_s * frame_rate_hz))
    t = np.arange(kernel_len) / frame_rate_hz
    kernel = np.exp(-t / tau_decay_s)
    kernel /= kernel.sum()

    # Convolve each neuron (FFT conv for speed)
    fluorescence = np.zeros_like(events, dtype=float)
    for n in range(num_neurons):
        fluorescence[:, n] = np.convolve(events[:, n], kernel, mode="same")

    fluorescence += baseline
    fluorescence += rng.normal(0.0, noise_sd, size=fluorescence.shape)

    # Add mild heterogeneity across neurons
    fluorescence *= rng.lognormal(mean=0.0, sigma=0.25, size=(1, num_neurons))

    return timestamps_s, fluorescence


def plot_neuron(
    timestamps_s: np.ndarray,
    fluorescence: np.ndarray,
    neuron_index: int,
    *,
    ax: plt.Axes | None = None,
    title: str | None = None,
) -> plt.Axes:
    """Plot one neuron's fluorescence over time."""
    if ax is None:
        _, ax = plt.subplots(figsize=(10, 3))
    ax.plot(timestamps_s, fluorescence[:, neuron_index])
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Fluorescence (a.u.)")
    ax.set_title(title or f"Neuron {neuron_index}")
    return ax


In [ ]:
timestamps_s, fluorescence = simulate_calcium_data(
    num_neurons=60,
    duration_s=90,
    frame_rate_hz=20,
    event_rate_hz=0.15,
    tau_decay_s=1.0,
    noise_sd=0.12,
    rng=rng,
)

num_samples, num_neurons = fluorescence.shape
print(f"samples={num_samples:,}  neurons={num_neurons}")

In [ ]:
# Quick look at a few neurons
for n in [0, 1, 2]:
    plot_neuron(timestamps_s, fluorescence, n)
plt.show()

# Part I: Core concepts

We'll move from common pain points in analysis scripts to small, composable functions.

## Roadmap
1. Naming
2. Modularity
3. Cyclomatic complexity + least surprise + loose coupling
4. Documentation (docstrings)
5. Handling errors
6. Test-driven development (TDD)


## 1. The art of naming

Names are the cheapest form of documentation. In practice, good names:
- Tell you **what** something represents (and units, if relevant).
- Tell you **why** it exists (purpose), not just its type.
- Make incorrect usage feel awkward.

We'll start with an intentionally unpleasant example.


In [ ]:
def fmp(x, y, t):
    """(Intentionally bad) Find neuron with most 'peaks' above a threshold.

    Problems (on purpose):
    - x, y, t don't communicate meaning
    - threshold and minimum separation are conflated
    - mixing lists + indexes makes off-by-one bugs easy
    """
    s = len(x)
    thr = t * np.std(y, axis=0)
    a = []
    for n in range(y.shape[1]):
        i = 0
        one = 0
        pk = thr[n]
        while i < s:
            if y[i, n] > pk:
                one += 1
                i += t  # <- wait... why are we stepping by the threshold?
            else:
                i += 1
        a.append(one)
    return a.index(max(a))


bad_answer = fmp(timestamps_s, fluorescence, 2)
bad_answer

Even if this runs, it's hard to *trust*:

- Is `t` a threshold? a stride? both?
- Are events counted correctly?
- What happens if we later change how thresholds are defined?

Let's rewrite this so the call site explains itself.


In [ ]:
def count_threshold_crossings(
    trace: np.ndarray,
    *,
    threshold: float,
    min_separation_samples: int,
) -> int:
    """Count threshold crossings with a simple refractory period.

    Notes
    -----
    This is a deliberately simple event detector for teaching purposes.
    For real event detection you'd likely use deconvolution or a validated method.
    """
    if min_separation_samples < 1:
        raise ValueError("min_separation_samples must be >= 1")

    count = 0
    i = 0
    n = trace.size
    while i < n:
        if trace[i] > threshold:
            count += 1
            i += min_separation_samples
        else:
            i += 1
    return count


def find_neuron_with_most_events(
    fluorescence: np.ndarray,
    *,
    num_std_dev: float = 2.0,
    min_separation_s: float = 0.25,
    frame_rate_hz: float = 20.0,
) -> int:
    """Return the neuron index with the most detected events.

    Parameters
    ----------
    fluorescence:
        Array of shape (num_samples, num_neurons).
    num_std_dev:
        Per-neuron threshold as `num_std_dev * std(trace)`.
    min_separation_s:
        Refractory period between events (seconds).
    frame_rate_hz:
        Sampling rate (Hz).

    Returns
    -------
    neuron_index:
        Integer index of the neuron with the most detected events.
    """
    if fluorescence.ndim != 2:
        raise ValueError("fluorescence must be 2D: (samples, neurons)")
    if num_std_dev <= 0:
        raise ValueError("num_std_dev must be > 0")
    if min_separation_s <= 0 or frame_rate_hz <= 0:
        raise ValueError("min_separation_s and frame_rate_hz must be > 0")

    min_sep = int(np.ceil(min_separation_s * frame_rate_hz))
    thresholds = num_std_dev * np.std(fluorescence, axis=0)

    counts = np.array(
        [
            count_threshold_crossings(
                fluorescence[:, n],
                threshold=thresholds[n],
                min_separation_samples=min_sep,
            )
            for n in range(fluorescence.shape[1])
        ]
    )
    return int(np.argmax(counts))


good_answer = find_neuron_with_most_events(
    fluorescence,
    num_std_dev=2.0,
    min_separation_s=0.25,
    frame_rate_hz=20.0,
)
good_answer

In [ ]:
# The call site is now self-documenting:
print("Bad answer:", bad_answer)
print("Good answer:", good_answer)

plot_neuron(timestamps_s, fluorescence, good_answer, title="Neuron with most detected events")
plt.show()

## 2. Modularity

A modular function is:
- **Small** enough to understand quickly,
- **Pure** when possible (few side effects),
- **Easy to test** in isolation,
- Reusable in other analyses.

A common failure mode is a single "kitchen sink" function that mixes:
data access → preprocessing → detection → statistics → plotting.


### A monolithic version (example to refactor)

Below is a plausible *anti-pattern*. It's here for reading, not for running.

```python
def calculate_iti_bad(timestamps_s, fluorescence, num_std_dev, min_separation):
    # mixes event finding, neuron looping, and summary statistics
    # easy to introduce bugs (e.g., neuron_index undefined)
    ...
```


In [ ]:
def find_event_indices(
    trace: np.ndarray,
    *,
    threshold: float,
    min_separation_samples: int,
) -> np.ndarray:
    """Return indices of detected events in a 1D trace."""
    indices = []
    i = 0
    n = trace.size
    while i < n:
        if trace[i] > threshold:
            indices.append(i)
            i += min_separation_samples
        else:
            i += 1
    return np.asarray(indices, dtype=int)


def event_times_from_indices(timestamps_s: np.ndarray, event_indices: np.ndarray) -> np.ndarray:
    """Map event sample indices to event times (seconds)."""
    return timestamps_s[event_indices]


def inter_event_intervals_s(event_times_s: np.ndarray) -> np.ndarray:
    """Compute inter-event intervals (seconds)."""
    if event_times_s.size < 2:
        return np.array([], dtype=float)
    return np.diff(event_times_s)


def mean_interval(intervals_s: np.ndarray) -> float:
    """Mean of intervals; returns NaN if no intervals."""
    return float(np.mean(intervals_s)) if intervals_s.size else float("nan")


In [ ]:
def mean_iti_for_neuron(
    timestamps_s: np.ndarray,
    fluorescence: np.ndarray,
    neuron_index: int,
    *,
    num_std_dev: float,
    min_separation_s: float,
    frame_rate_hz: float,
) -> float:
    """Mean inter-event interval (ITI) for one neuron."""
    if not (0 <= neuron_index < fluorescence.shape[1]):
        raise IndexError("neuron_index out of range")

    min_sep = int(np.ceil(min_separation_s * frame_rate_hz))
    threshold = float(num_std_dev * np.std(fluorescence[:, neuron_index]))

    event_idx = find_event_indices(
        fluorescence[:, neuron_index],
        threshold=threshold,
        min_separation_samples=min_sep,
    )
    event_t = event_times_from_indices(timestamps_s, event_idx)
    itis = inter_event_intervals_s(event_t)
    return mean_interval(itis)


def neuron_with_smallest_mean_iti(
    timestamps_s: np.ndarray,
    fluorescence: np.ndarray,
    *,
    num_std_dev: float = 2.0,
    min_separation_s: float = 0.25,
    frame_rate_hz: float = 20.0,
) -> int:
    """Return neuron index with smallest mean ITI (fastest event rate)."""
    means = np.array(
        [
            mean_iti_for_neuron(
                timestamps_s,
                fluorescence,
                n,
                num_std_dev=num_std_dev,
                min_separation_s=min_separation_s,
                frame_rate_hz=frame_rate_hz,
            )
            for n in range(fluorescence.shape[1])
        ],
        dtype=float,
    )
    # Treat NaNs (no events) as +inf so they don't win
    means = np.where(np.isnan(means), np.inf, means)
    return int(np.argmin(means))


fastest_neuron = neuron_with_smallest_mean_iti(timestamps_s, fluorescence)
fastest_neuron

In [ ]:
plot_neuron(timestamps_s, fluorescence, fastest_neuron, title="Neuron with smallest mean ITI")
plt.show()

Notice what modularity buys you:
- Each helper is independently testable.
- You can swap out `find_event_indices` without rewriting the statistics.
- Bugs are localized to a small surface area.


## 3. Complexity, least surprise, and loose coupling

### Cyclomatic complexity
Cyclomatic complexity is (roughly) the number of independent paths through a function.
High complexity often comes from many branches and deep nesting — which makes testing harder.

### Principle of least surprise
Try to make functions behave the way a reasonable reader expects:
- clear parameter names
- predictable return types
- no hidden global state
- no silent unit conversions

### Loose coupling
Prefer passing explicit inputs to a function over relying on global variables or I/O inside the function.


In [ ]:
# Example: surprising behavior due to hidden global state
GLOBAL_THRESHOLD_STD = 2.0  # <- don't do this in real analysis code


def count_events_surprising(trace: np.ndarray) -> int:
    """Count events above GLOBAL_THRESHOLD_STD * std(trace). (Surprising!)"""
    thr = GLOBAL_THRESHOLD_STD * np.std(trace)
    return count_threshold_crossings(trace, threshold=thr, min_separation_samples=5)


def count_events_explicit(trace: np.ndarray, *, num_std_dev: float, min_separation_samples: int) -> int:
    """Count events above num_std_dev * std(trace). (Explicit.)"""
    thr = float(num_std_dev * np.std(trace))
    return count_threshold_crossings(trace, threshold=thr, min_separation_samples=min_separation_samples)


trace0 = fluorescence[:, 0]
print("surprising:", count_events_surprising(trace0))
print("explicit:", count_events_explicit(trace0, num_std_dev=2.0, min_separation_samples=5))


In [ ]:
# Example: reduce nesting by using early returns (often lowers complexity)

def classify_trace_quality_bad(trace: np.ndarray) -> str:
    if trace.size > 0:
        if np.isfinite(trace).all():
            if np.std(trace) > 0:
                if np.max(trace) < 10:
                    return "ok"
                else:
                    return "saturated"
            else:
                return "flat"
        else:
            return "nan_or_inf"
    else:
        return "empty"


def classify_trace_quality(trace: np.ndarray) -> str:
    if trace.size == 0:
        return "empty"
    if not np.isfinite(trace).all():
        return "nan_or_inf"
    if np.std(trace) == 0:
        return "flat"
    if np.max(trace) >= 10:
        return "saturated"
    return "ok"


print(classify_trace_quality_bad(trace0), classify_trace_quality(trace0))


## 4. Documentation

Docstrings are for **external communication**:
- What the function does (one sentence).
- Parameters (type, meaning, units).
- Returns (shape/type, meaning).
- Assumptions and edge cases.

Inline comments are for **local clarification** of non-obvious steps.


In [ ]:
def zscore_per_neuron(fluorescence: np.ndarray) -> np.ndarray:
    """Z-score each neuron independently.

    Parameters
    ----------
    fluorescence:
        Array of shape (num_samples, num_neurons).

    Returns
    -------
    z:
        Z-scored array of the same shape as `fluorescence`.
        Each column has mean 0 and std 1 (unless std was 0, in which case the column is zeros).
    """
    if fluorescence.ndim != 2:
        raise ValueError("fluorescence must be 2D: (samples, neurons)")

    mu = np.mean(fluorescence, axis=0, keepdims=True)
    sd = np.std(fluorescence, axis=0, keepdims=True)
    sd_safe = np.where(sd == 0, 1.0, sd)
    z = (fluorescence - mu) / sd_safe
    # If sd was 0, define the result as 0 rather than NaN/Inf
    z = np.where(sd == 0, 0.0, z)
    return z


z = zscore_per_neuron(fluorescence)
z.shape, float(np.mean(z[:, 0])), float(np.std(z[:, 0]))


## 5. Handle errors (and make misuse loud)

Two pragmatic rules for analysis code:
1. **Validate inputs at the boundary** (types/shapes/ranges).
2. When something is wrong, fail **early** with an error message that tells you what to fix.


In [ ]:
def validate_fluorescence_matrix(fluorescence: np.ndarray) -> None:
    """Validate that fluorescence looks like (samples, neurons) numeric data."""
    if not isinstance(fluorescence, np.ndarray):
        raise TypeError("fluorescence must be a NumPy array")
    if fluorescence.ndim != 2:
        raise ValueError(f"fluorescence must be 2D, got shape {fluorescence.shape}")
    if fluorescence.shape[0] < 2:
        raise ValueError("need at least 2 samples")
    if fluorescence.shape[1] < 1:
        raise ValueError("need at least 1 neuron")
    if not np.isfinite(fluorescence).all():
        raise ValueError("fluorescence contains NaN or Inf")


# Demo: comment/uncomment to see a useful failure
validate_fluorescence_matrix(fluorescence)
print("validation ok")

## 6. Test-driven development (TDD)

A lightweight TDD loop:
1. Write a test describing desired behavior (**fails**).
2. Write the simplest code to make it pass.
3. Refactor while keeping tests green.

In notebooks you can start with `assert` tests. For real projects, move tests into `pytest`.


In [ ]:
# Minimal "notebook tests" for a couple of helpers

# count_threshold_crossings
trace = np.array([0.0, 0.2, 1.5, 0.1, 2.0, 0.0])
assert count_threshold_crossings(trace, threshold=1.0, min_separation_samples=1) == 2
assert count_threshold_crossings(trace, threshold=1.0, min_separation_samples=2) == 2

# inter_event_intervals_s
assert np.allclose(inter_event_intervals_s(np.array([0.0, 0.5, 2.0])), np.array([0.5, 1.5]))
assert inter_event_intervals_s(np.array([0.0])).size == 0

print("basic tests passed")

### What a pytest test might look like

```python
# test_events.py
import numpy as np
from your_module import count_threshold_crossings

def test_count_threshold_crossings():
    trace = np.array([0.0, 0.2, 1.5, 0.1, 2.0, 0.0])
    assert count_threshold_crossings(trace, threshold=1.0, min_separation_samples=1) == 2
```

From the terminal:

```bash
pytest -q
```


# Part II: Modern tooling (fast, practical)

You can write clean code **without** tools — but tools make it much easier to stay clean as a project grows.

## Suggested minimal stack
- **IDE**: PyCharm or VS Code (autocomplete, jump-to-definition, refactor tools).
- **Formatter + linter**: `ruff` (format + lint).
- **Tests**: `pytest`.
- **Project/deps**: `uv` (fast environment + dependency management) *or* `pip + venv`.
- **Automation**: `pre-commit` to run formatting/lint/tests before commits.

## Typical commands (example)
```bash
# create env + install deps (one approach)
uv venv
uv pip install -r requirements.txt

# format + lint
ruff format .
ruff check .

# run tests
pytest -q
```

## Notebook hygiene tips
- Keep heavy logic in `.py` modules; notebooks should orchestrate + visualize.
- Prefer small, reusable functions (then test them).
- If you share notebooks, run them top-to-bottom before publishing.


## Exercises (10–15 min)
1. Modify `find_event_indices` to require a *rising-edge* crossing (to avoid counting sustained plateaus).
2. Write tests that cover:
   - no events,
   - events exactly at threshold,
   - extreme min_separation_samples.
3. Refactor `find_neuron_with_most_events` so thresholds are computed by an injectable function
   (e.g., `threshold_fn(trace) -> float`).
